<a href="https://colab.research.google.com/github/agnesedaff/Implicit_obj_completion/blob/main/IMPLICIT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install transformers # installiamo la libreria transformers di GitHub
!python -m pip install spacy # installiamo SpaCy per POS tagging, similarity e lemmatizzazione
!python -m spacy download it_core_news_lg # scarichiamo il modello per l'italiano da usare con SpaCy

2024-01-23 15:03:08.371319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 15:03:08.371381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 15:03:08.373190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 15:03:10.169030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [ ]:
from transformers import pipeline # importiamo il modulo pipeline
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased', tokenizer= 'bert-base-multilingual-cased') # Inizializziamo la pipeline per il task fill-mask con il modello che vogliamo (qui la demo con BERT base)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
import spacy
nlp = spacy.load("it_core_news_lg")

In [ ]:
documento = open('/content/drive/MyDrive/implicit_obj_dataset.txt','r') # apriamo il documento con il dataset IMPLICIT da drive
sentences = documento.readlines() # splittiamo riga per riga

**Primo step:** otteniamo i risultati dei modelli sul task di fill-mask

Le istruzioni per il modello sono:

1.   riempi il primo [MASK] con un Determiner e crea una nuova frase con un solo [MASK]
2.   riempi il secondo [MASK] con un Noun

In [ ]:
output = []
scores = []

for sentence in sentences: # iteriamo su ogni frase nel dataset

    results = unmasker(sentence, top_k=10) # settiamo il numero di predizioni a 10
    det_predictions = []
    for prediction in results[0]: # cerchiamo se è disponibile un Determiner per il primo [MASK]
        doc = nlp(prediction["token_str"]) # avviamo l'analisi della parola con SpaCy
        for token in doc:
            if token.pos_ == "DET":
                det_predictions.append(prediction) # aggiungiamo il token con POS = DET più probabile alla variabile 'det_predictions'
    if det_predictions:
            det_predictions.sort(key=lambda x: x["score"], reverse=True)
            determiner = det_predictions[0]["token_str"]
            new_sentence = sentence.replace("[MASK]", determiner, 1) # sostituiamo il Determiner al primo [MASK] e creiamo una nuova frase con un solo [MASK]
    else:
            new_sentence = sentence.replace("[MASK]", "", 1)

    results_new = unmasker(new_sentence, top_k=20) # cerchiamo un Noun per il secondo [MASK]
    noun_predictions = []
    for prediction in results_new:
        doc = nlp(prediction['token_str'])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output.append(noun_predictions[0]["token_str"]) # aggiungiamo il token con POS = NOUN più probabile alla variabile 'noun_predictions'
        scores.append(noun_predictions[0]["score"]) # aggiungiamo il prediction score alla variabile 'scores'
    else:
        output.append("unknown")
        scores.append("unknown")
scores_clean = [] # arrotondiamo a due decimali i prediction scores
for score in scores:
  if score == 'unknown':
    scores_clean.append('unknown')
  else:
    scores.append(round(score, 2))

print('predicted_words =', output)
print('prediction_scores =', scores.clean)

**Secondo step**: calcolo della similarity tra gli annotatori e i risultati dei modelli

In [ ]:
# riportiamo i risultati di tutti i modelli dopo aver iterato il passaggio precedente

bertm = ['piano', 'voci', 'nemico', 'suono', 'concerto', 'brano', 'parole', 'musica', 'pianoforte', 'lingua', 'tradizione', 'nemico', 'unknown', 'suono', 'suono', 'scena', '##nari', 'uomo', 'scena', 'musica', 'giorno', 'fase', 'fine', 'festa', 'minuto', 'calcio', 'mondo', 'verbo', 'palco', 'giorno', 'attacco', 'morte', 'vita', 'messa', 'comando', 'linea', 'punto', 'incidente', 'partita', 'manifestazione', 'vino', 'vino', 'fibre', 'mani', 'unknown', 'carne', 'braccio', 'oggetto', 'sangue', 'gola', 'vino', '##vento', 'vino', 'giorno', 'mani', 'uomo', 'nome', 'vino', 'casa', 'pelle', 'canzone', 'grazie', 'canzone', 'unknown', 'testo', 'brano', 'volte', 'giorno', 'canzone', 'canto', 'canzone', 'canzone', 'piano', 'musica', 'canto', 'canzone', 'piano', 'pianoforte', 'canzone', 'notte', 'mondo', 'mare', 'chiave', 'paese', 'stazione', 'detective', 'casa', 'giorno', 'strada', 'ragazza', 'ragazzi', 'mano', 'futuro', 'mondo', 'casa', 'quartiere', 'strada', 'sistema', 'pilota', 'luogo', 'unknown', 'guerra', '#', 'guerra', 'guerra', 'campione', 'mondo', 'guerra', 'nemico', 'fuoco', 'combattimento', 'giorno', 'guerra', 'fuoco', 'conflitto', 'mondo', 'riconoscimento', 'campioni', 'nemico', 'nemico', 'moglie', '##nti', 'voce', '##re', 'voce', 'cantante', 'giornalista', '##nti', '##nti', '##nti', '##nti', '##re', 'direzione', 'Presidente', '##nti', 'giornalista', '##trice', 'percorso', 'tenente', '##nti', 'libertà', 'cose', 'motore', 'energia', 'moda', 'tempo', 'energia', '##utta', 'forza', 'vita', 'lot', 'peso', '##utta', 'denaro', '##utta', 'futuro', 'risorse', '##utta', 'euro', 'forza', 'campionato', 'pista', 'ciclismo', 'percorso', 'domenica', 'giro', 'contratto', 'stagione', 'strada', 'strada', 'mondo', 'moto', 'pilota', 'distanza', 'moto', 'unknown', '##dosi', 'gara', 'mail', 'strada', 'oggetto', 'carne', 'sala', 'cena', 'piante', 'unknown', 'casa', 'mondo', 'bambini', 'festa', 'carne', 'musica', 'vino', 'tavola', 'notte', 'casa', 'festa', 'volte', 'carne', 'carta', '##re', 'musica', 'regista', 'attori', 'film', 'film', 'film', 'scena', '##re', 'orchestra', 'regia', '##re', 'film', 'film', '##ing', 'programma', 'coro', 'film', 'gara', 'commedia', 'scuola', 'musica', 'lavoro', 'rivista', 'tela', 'fotografia', 'disegno', 'serie', 'anima', 'disegno', 'albo', 'disegno', 'disegni', 'piano', 'romanzi', 'ambiente', 'tela', 'personaggi', 'film', 'storia', 'padre', 'cielo', 'unknown', 'casa', 'notte', 'lavoro', 'specie', 'sangue', 'luce', 'giorno', 'fuoco', 'terra', 'casa', 'legge', 'unknown', 'giornale', 'notte', 'parete', 'bar', 'lettera', 'partita', 'gioco', 'campionato', 'sera', 'derby', 'gioco', 'gioco', 'gioco', 'tempo', 'calcio', 'partita', 'campo', 'partita', 'gioco', 'match', 'ritorno', 'campionato', 'tennis', 'gioco', 'partita', 'titolo', 'lavoro', 'punteggio', 'possibilità', 'lavoro', 'lavoro', 'Croce', 'titolo', 'posizione', 'avversario', '##mento', 'tempo', 'tempo', 'metà', 'collaborazione', 'software', 'mercato', 'denaro', 'punteggio', 'premio', 'taxi', 'paese', 'aereo', 'strada', 'gara', 'ponte', 'circuito', 'strada', 'unknown', 'regole', 'gioco', 'mondo', 'taxi', 'mondo', 'treno', 'rete', 'cuore', 'bambini', 'aereo', 'mondo', 'bambino', 'note', 'Forum', 'libro', 'testo', 'pianoforte', 'unknown', 'latino', 'mani', 'giorno', 'libro', 'unknown', 'dialetto', 'libro', 'parola', 'articolo', 'cose', 'unknown', 'testi', 'testo', 'carne', 'madre', 'passione', 'cibo', '##re', 'giorno', 'vita', 'giorno', 'carne', 'carne', 'cibo', '##΄', 'carne', 'mano', 'vino', 'casa', 'cibo', 'musica', 'carne', 'vino', 'unknown', 'giorno', 'bar', 'Signore', 'URL', 'strada', 'conto', 'servizio', 'gare', 'attività', 'gioco', '##tura', 'portale', 'professionista', 'passaggio', 'ordine', 'corse', 'lavoro', 'cibo', 'giorno', 'parola', 'prezzo', 'prezzo', 'denaro', 'cane', 'tributo', 'legge', 'metà', 'terra', 'trasferimento', 'lavoro', 'unknown', 'libertà', 'gruppo', 'guerra', 'prezzo', 'valore', 'giorno', 'battaglia', 'pagine', 'club', 'vantaggio', 'calcio', 'avversario', 'partita', 'punteggio', 'derby', 'amici', 'partita', 'gol', 'gioco', 'gioco', 'Paese', 'classifica', 'partita', 'derby', 'campionato', 'limite', 'potere', 'mondo', 'salute', 'vita', 'Signore', 'Signore', 'unknown', 'anno', 'musica', 'vino', 'Chiesa', 'campo', 'cose', 'gente', 'unknown', 'attenzione', 'titolo', 'cura', 'diritto', 'necessità', 'gruppo', 'necessità', 'popolazione', 'unknown', 'unknown', 'pensiero', 'qualità', 'società', 'riguardo', 'situazione', 'mondo', 'tempo', 'Governo', 'volta', 'unknown', 'violenza', 'popolazione', 'momento', 'società', 'riguardo', 'unknown', 'unknown', 'musica', 'musica', 'testo', 'passione', 'musica', 'tempo', 'luoghi', 'musica', 'torre', 'nome', 'mondo', 'musica', 'musica', 'gioco', 'spettacolo', 'musica', 'stagione', 'Piano', 'piano', 'spettacolo', 'verità', 'pratica', 'unknown', 'vita', 'giorno', 'unknown', 'corpo', 'città', 'corpo', 'unknown', 'femmine', '##re', 'mondo', '##re', 'scala', 'corpo', 'unknown', 'terra', 'occhi', 'corpo', 'codice', 'lettera', 'tesi', 'libro', 'unknown', 'articolo', 'testo', 'articoli', 'conto', 'articolo', 'romanzo', 'codice', 'giornale', 'opere', 'testo', 'musica', 'unknown', 'articolo', 'codice', 'testo', 'messa', 'giorno', 'unknown', 'gioco', 'lavoro', 'fuoco', 'luna', 'carne', 'famiglia', 'anno', 'voce', 'messa', 'cibo', 'vino', 'materiale', 'gruppo', 'carne', 'vino', 'vino', 'notte', 'pianoforte', 'chitarra', 'jazz', 'pianoforte', 'pelle', 'concerto', 'brano', 'unknown', 'pianoforte', 'brani', 'notte', 'concerto', 'pianoforte', 'jazz', 'sera', 'musica', 'pianoforte', 'musica', 'musica', 'coro', 'gara', 'gol', 'calcio', 'colpo', 'difensore', 'cross', 'linea', 'pistola', 'testa', 'segno', 'salto', 'gol', 'mani', 'fuoco', 'calcio', 'giocatore', 'gol', 'calcio', 'giocatori', 'mano', 'gioco', 'classifica', 'mondo', 'collegio', 'gioco', 'gara', 'mondo', 'gioco', 'partita', 'premio', 'campionato', 'partita', 'girone', 'titolo', 'corsa', 'campionato', 'vittoria', 'guerra', 'vittoria', 'successo']
bert_it = ['brano', 'silenzio', 'musica', 'brano', 'concerto', 'brano', 'brano', 'musica', 'canzoni', 'lingua', 'giudice', 'voce', 'mantello', 'canto', 'video', 'discorso', 'pianoforte', 'mondo', 'musica', 'silenzio', 'attimo', 'giorno', 'giorno', 'momento', 'tempo', 'mercato', 'sole', 'attimo', 'personaggio', 'unknown', 'controllo', 'momento', 'ritorno', 'minuto', 'notte', 'volo', 'dovere', 'sole', 'treno', 'sospensione', 'vino', 'vino', 'acqua', 'vino', 'alcolici', 'vino', 'sangue', 'caffè', 'bicchiere', 'caffè', 'vino', 'vino', 'vino', 'vino', 'coppa', 'vino', 'caffè', 'vino', 'acqua', 'drink', 'canzoni', 'pezzo', 'brano', 'unknown', 'coro', 'brano', 'brano', 'pezzo', 'brano', 'inno', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'blues', 'blues', 'canzone', 'numero', 'Comune', 'SMS', 'sito', 'telefono', 'professionisti', 'telefono', 'prodotto', 'Comune', 'ragazza', 'medico', 'macchina', 'telefono', 'telefono', 'negozio', 'telefono', 'macchina', 'ministero', 'pilota', 'numero', 'unknown', 'combattimento', 'guerra', 'giorno', 'battaglia', 'concorrenza', 'diavolo', 'guerra', 'volta', 'nemico', 'guerra', 'guerra', 'guerra', 'guerra', 'piede', 'anno', 'lotta', 'match', 'problema', 'battaglia', 'dibattito', 'serata', 'serata', 'conduttore', 'giornalista', 'cantautore', 'conduttore', 'giornalista', 'giornalista', 'Maestro', 'giornalista', 'dibattito', 'giornalista', 'dibattito', 'concorrente', 'giornalista', 'serata', 'dibattito', 'giornalista', 'giornalista', 'cibo', 'soldi', 'motore', 'pianeta', 'moda', 'grassi', 'risorse', 'energia', 'cibo', 'cibo', 'giorno', 'emissioni', 'grassi', 'denaro', 'energia', 'energia', 'terra', 'energia', 'euro', 'carburante', 'gara', 'anno', 'ciclismo', 'gara', 'domenica', 'giro', 'corsa', 'anno', 'gara', 'unknown', 'meeting', 'unknown', 'titolo', 'distanza', 'testa', 'titolo', 'corsa', 'tempo', 'batteria', 'piano', 'piatto', 'pollo', 'cibo', 'pasta', 'piatto', 'piatto', 'piatto', 'pesce', 'pasto', 'pesce', 'piatto', 'pane', 'cibi', 'pasta', 'pollo', 'pranzo', 'pasta', 'cibo', 'cibi', 'unknown', 'coro', 'pianoforte', 'mondo', 'produzione', 'produttori', 'film', 'film', 'concerti', '##nti', 'concerto', 'film', 'coro', 'progetto', 'film', 'progetto', 'concerto', 'barca', 'film', 'gioco', 'film', 'disegno', 'quadri', 'disegno', 'disegno', 'quadro', 'fumetti', 'disegno', 'orso', 'immagini', 'disegno', 'fumetto', 'ombre', 'disegni', 'piano', 'disegno', 'paesaggio', 'composizione', 'ritratti', 'personaggio', 'personaggio', 'tabacco', 'tabacco', 'unknown', 'tabacco', 'sigaretta', 'tabacco', 'sigaretta', 'sigaretta', 'tabacco', 'sigaretta', 'tabacco', 'sigaretta', 'caffè', 'fumo', 'fumo', 'sigaretta', 'tabacco', 'caffè', 'sigaretta', 'tabacco', 'partita', 'partita', 'campionato', 'match', 'partita', 'squadra', 'giorno', 'modalità', 'calcio', 'partita', 'partita', 'pallone', 'partita', 'ruolo', 'match', 'partita', 'torneo', 'tempo', 'match', 'calcio', 'soldo', 'salute', 'euro', 'vita', 'denaro', 'denaro', 'euro', 'pane', 'lavoro', 'centesimo', 'percentuali', 'soldi', 'denaro', 'punti', 'lavoro', 'denaro', 'soldo', 'soldo', 'soldo', 'soldo', 'macchina', 'moto', 'veicolo', 'macchina', 'moto', 'veicolo', 'vettura', 'macchina', 'macchina', 'responsabilità', 'veicolo', 'macchina', 'ragazzi', 'macchina', 'veicolo', 'macchina', 'cuore', 'unknown', 'moto', 'macchina', 'testo', 'brano', 'testi', 'giornale', 'messaggio', 'libri', 'libro', 'storia', 'carta', 'storia', 'libro', 'libro', 'libro', 'testa', 'parola', 'libri', 'sito', 'fumetto', 'giornali', 'resoconto', 'uovo', 'latte', 'pane', 'pesce', 'cibo', 'casa', 'cibo', 'pasto', 'unknown', 'pranzo', 'panino', 'pane', 'pesce', 'panino', 'carne', 'pasti', 'panino', 'pizza', 'pranzo', 'pane', 'INFORMAZIONI', 'caffè', 'caffè', 'caffè', 'prodotto', 'caffè', 'conto', 'prodotto', 'prodotti', 'prodotti', 'gioco', 'copia', 'corso', 'caffè', 'caffè', 'cibo', 'pranzo', 'pasto', 'cibo', 'caffè', 'prezzo', 'vita', 'cittadinanza', 'multa', 'prezzo', 'tassa', 'multa', 'vita', 'vita', 'perdita', 'riscatto', 'conto', 'vita', 'prezzo', 'danni', 'presidente', 'unknown', 'carcere', 'colpo', 'vita', 'treno', 'partita', 'controllo', 'partita', 'partita', 'partita', 'partita', 'testa', 'cappotto', 'partita', 'controllo', 'vita', 'democrazia', 'partita', 'partita', 'partita', 'campionato', 'pallone', 'strada', 'controllo', 'giorno', 'Signore', 'misericordia', 'preghiera', 'Signore', 'unknown', 'preghiera', 'Signore', 'Signore', 'Signore', 'lavoratori', 'Dio', 'cortesia', 'clientela', 'medico', 'direzione', 'cortesia', 'cittadinanza', 'Sindaco', 'docente', 'medici', 'popolazione', 'giocatore', 'rischio', 'giuria', 'Camera', 'problema', 'collettività', 'realtà', 'paziente', 'preoccupazione', 'mente', '##tivo', 'criminalità', 'Paese', 'rischio', 'lettori', 'ricercatori', 'governo', 'situazione', 'materiale', 'musica', 'biglietto', 'preparazione', 'pezzo', 'musica', 'luogo', 'musica', 'stanza', 'prodotto', 'rock', 'batteria', 'musica', 'passione', 'teatro', 'gioco', 'unknown', 'drink', 'tiro', 'teatro', 'unknown', 'bocca', 'luogo', 'respiro', 'mondo', 'respiro', 'mare', 'città', 'respiro', 'respiro', 'corpo', 'respiro', 'respiro', 'sangue', 'respiro', 'corpo', 'veicolo', 'mare', 'respiro', 'odore', 'testo', 'libro', 'mail', 'messaggio', 'unknown', 'articolo', 'commento', 'verità', 'testo', 'notizie', 'articolo', 'testo', 'libro', 'romanzo', 'lettera', 'messaggio', 'unknown', 'lettera', 'poesie', 'libro', 'piatto', 'caffè', 'cibo', 'clienti', 'piatti', 'cena', 'cibo', 'clienti', 'clienti', 'birra', 'caffè', 'pesce', 'piatto', 'pollo', 'piatto', 'cibo', 'piatto', 'piatto', 'piatto', 'cibo', 'disco', 'anno', 'concerto', 'violino', 'pianoforte', 'rock', 'unknown', 'piano', 'sveglia', 'chitarra', 'sveglia', 'pianoforte', 'chitarra', 'unknown', 'violino', 'chitarra', 'ritmo', 'cori', 'violino', 'violino', 'fiato', 'pallone', 'fiato', 'palla', 'tiro', 'tiro', 'pallone', 'palla', 'palla', 'pallone', 'palla', 'pallone', 'decina', 'tiro', 'pallone', 'palla', 'tiro', 'tiro', 'palla', 'palla', 'premio', 'partita', 'premio', 'concorso', 'premio', 'gara', 'guerra', 'partita', 'partita', 'premio', 'campionato', 'partita', 'partita', 'gara', 'Coppa', 'campionato', 'guerra', 'guerra', 'vittoria', 'vittoria']
umberto = ['brano', 'silenzio', 'discorso', 'discorso', 'brano', 'rock', 'giro', 'musica', 'brano', 'testo', 'popolo', 'discorso', 'voce', 'notizia', 'brano', 'discorso', 'pianoforte', 'urla', 'discorso', 'mondo', 'unknown', 'giorno', 'giorno', 'giorno', 'pagina', 'futuro', 'lavoro', 'articolo', 'momento', 'unknown', 'controllo', 'bambino', 'viaggio', 'passaggio', 'paziente', 'messa', 'momento', 'unknown', 'pullman', 'discussione', 'uscita', 'vino', 'acqua', 'vino', 'latte', 'vino', 'sangue', 'caffè', 'bevanda', 'latte', 'vino', 'vino', 'vino', 'pane', 'unknown', 'vino', 'caffè', 'vino', 'acqua', 'caffè', 'canzone', 'canzone', 'canzone', 'unknown', 'tifo', 'canto', 'volte', 'messaggio', 'brano', 'inno', 'canzone', 'brano', 'canzone', 'musica', 'canzone', 'tromba', 'unknown', 'tromba', 'canzone', 'canzone', 'segreteria', 'familiari', 'Wind', 'concessionario', 'linea', 'rivenditore', 'telefono', 'posta', 'telefono', 'ragazza', 'medico', 'polizia', 'numero', 'appuntamento', 'venditore', 'telefono', 'polizia', 'ministero', 'capitano', 'numero', 'unknown', 'combattimento', 'caccia', 'giorno', 'giorno', 'titolo', 'Signore', 'giorno', 'battaglia', 'nemico', 'terrorismo', 'anno', 'giorno', 'guerra', 'battaglia', 'paura', 'lotta', 'unknown', 'giorno', 'battaglia', 'convegno', 'trasmissione', 'presentazione', 'conduttore', 'dibattito', 'gruppo', 'giornalista', 'giornalista', 'giornalista', 'maestro', 'giornalista', 'dibattito', 'giornalista', 'dibattito', 'conduttrice', 'giornalista', 'giornalista', 'dibattito', 'giornalista', 'giornalista', 'cibo', 'denaro', 'motore', 'cibo', 'critica', 'energia', 'cibo', 'processore', 'calorie', 'cibo', 'suolo', 'carburante', 'cibo', 'tempo', 'energia', 'petrolio', 'risorse', 'energia', 'carburante', 'carburante', 'rischio', 'campionato', 'ciclismo', 'giorno', 'domenica', 'campionato', 'unknown', 'anno', 'maratona', 'giorno', 'torneo', 'domenica', 'mondo', 'distanza', 'rischi', 'trofeo', 'maratona', 'miglio', 'serie', 'gara', 'cose', 'pasta', 'pranzo', 'verdura', 'pasta', 'pasta', 'cibo', 'peperoni', 'pietanze', 'pesce', 'carne', 'cena', 'pesce', 'pasta', 'cibi', 'verdure', 'pasta', 'cibo', 'pietanze', 'unknown', 'maestro', 'coro', 'film', 'film', 'film', 'film', 'film', 'tata', 'serata', 'banda', 'scena', 'maestro', 'regista', 'film', 'collezione', 'direttore', 'processione', 'film', 'concerto', 'film', 'città', 'carta', 'disegno', 'fumetto', 'quadro', 'natura', 'colore', 'gatto', 'animazioni', 'disegno', 'fumetto', 'figure', 'quadro', 'piano', 'personaggio', 'film', 'immagini', 'ritratti', 'personaggio', 'personaggio', 'tabacco', 'tabacco', 'unknown', 'tabacco', 'sigaretta', 'tabacco', 'camere', 'droga', 'fumatori', 'notte', 'tabacco', 'sigaretta', 'sigaretta', 'tabacco', 'tabacco', 'sigaretta', 'sigaretta', 'tabacco', 'testa', 'sigaretta', 'titolare', 'partita', 'campionato', 'campionato', 'partita', 'squadra', 'giorno', 'giorno', 'match', 'gara', 'partita', 'match', 'partita', 'partita', 'partita', 'partita', 'gara', 'vittoria', 'set', 'calcio', 'bitcoin', 'denaro', 'guadagno', 'soldi', 'denaro', 'denaro', 'online', 'online', 'tempo', 'cazzo', 'presenza', 'soldi', 'denaro', 'salario', 'lavoro', 'online', 'euro', 'brevetto', 'mondo', 'denaro', 'macchina', 'macchina', 'veicolo', 'macchina', 'veicolo', 'uomo', 'vettura', 'automobilisti', 'macchina', 'bambino', 'ragazzo', 'mondo', 'macchina', 'vano', 'veicolo', 'auto', 'macchina', 'ggiando', 'auto', 'auto', 'storia', 'brano', 'topic', 'mondo', 'lettere', 'cielo', 'terra', 'giornale', 'giornale', 'lettere', 'libro', 'giornale', 'latino', 'poesie', 'parole', 'cose', 'regolamento', 'articolo', 'notizie', 'titolo', 'pasto', 'cucciolo', 'pane', 'riso', 'cane', 'sabbia', 'carne', 'pasto', 'unknown', 'cibo', 'pizza', 'cibo', 'pesce', 'formaggio', 'carne', 'giorno', 'sera', 'panino', 'pranzo', 'pane', 'FOTO', 'libro', 'cibo', 'pranzo', 'prodotto', 'caffè', 'conto', 'prodotto', 'prodotto', 'catalogo', 'giochi', 'libro', 'corsi', 'caffè', 'caffè', 'piatto', 'pranzo', 'gelato', 'bis', 'caffè', 'matrimonio', 'pizzo', 'tasse', 'prezzo', 'prezzo', 'tasse', 'pizzo', 'biglietto', 'pena', 'danno', 'riscatto', 'sconfitta', 'guerra', 'prezzo', 'tasse', 'prezzo', 'unknown', 'pena', 'guerra', 'vita', 'primato', 'partita', 'partita', 'controllo', 'partita', 'partita', 'derby', 'testa', 'partita', 'match', 'conto', 'vita', 'guerra', 'partita', 'coppa', 'unknown', 'campionato', 'concentrazione', 'consenso', 'testa', 'giorno', 'significato', 'misericordia', 'giorno', 'Signore', 'giorno', 'Vergine', 'Signore', 'Signore', 'unknown', 'Signore', 'pace', 'Cliente', 'cliente', 'prenotazione', 'lettore', 'unknown', 'Sindaco', 'Sindaco', 'docente', 'ricercatori', 'popolazione', 'unknown', 'rilievo', 'giuria', 'Commissione', 'ministro', 'consumatori', 'situazione', 'salute', 'rilievo', 'Chiesa', 'attimo', 'criminalità', 'premier', 'rilievo', 'stampa', 'scena', 'cuore', 'scena', 'materiale', 'note', 'idea', 'bomba', 'dischi', 'sensazione', 'unknown', 'musica', 'mmo', 'canzoni', 'band', 'vita', 'lezioni', 'idea', 'spettacolo', 'scena', 'prodotto', 'tecniche', 'giro', 'musical', 'unknown', 'sudore', 'terra', 'liquido', 'vita', 'respiro', 'silenzio', 'pianeta', 'respiro', 'respiro', 'maschi', 'palco', 'respiro', 'sangue', 'sangue', 'rumore', 'sudore', 'mare', 'silenzio', 'odore', 'testo', 'libro', 'articolo', 'amico', 'giorno', 'articolo', 'commento', 'articolo', 'lettera', 'articolo', 'articolo', 'lettere', 'blog', 'opere', 'lettera', 'messaggio', 'mail', 'lettera', 'libro', 'pensiero', 'portata', 'caffè', 'cibo', 'cibo', 'cibo', 'motivo', 'cibo', 'clienti', 'cibo', 'clienti', 'pesce', 'pranzo', 'piatto', 'piatto', 'piatto', 'cibo', 'piatto', 'piatto', 'piatto', 'cena', 'disco', 'anno', 'pianoforte', 'cd', 'voce', 'tour', 'piano', 'pianoforte', 'telefono', 'disco', 'stanza', 'disco', 'musica', 'batteria', 'pianoforte', 'chitarra', 'sveglia', 'chitarra', 'tromba', 'violino', 'rigore', 'pallone', 'cinghia', 'pallone', 'pallone', 'pallone', 'pallone', 'palla', 'palla', 'tiro', 'pallone', 'pallone', 'palla', 'tiro', 'pallone', 'palla', 'tiro', 'tiro', 'pallone', 'pallina', 'premio', 'partita', 'concorso', 'ballottaggio', 'premio', 'gara', 'centrosinistra', 'gioco', 'partita', 'unknown', 'derby', 'partita', 'partita', 'combattimento', 'unknown', 'campionato', 'guerra', 'referendum', 'battaglia', 'referendum']
bertino = ['recensione', 'musica', 'dottore', 'canto', 'brano', 'brani', 'canto', 'voce', 'canzoni', 'conversazioni', 'verdetto', 'nemico', 'passo', 'telefono', 'filmato', 'versi', 'pianoforte', 'urla', 'concerto', 'voci', 'minuto', 'minuto', 'minuto', 'avvenimento', 'unknown', 'minuto', 'giorno', 'verdetto', 'paziente', 'unknown', 'pattuglia', 'morte', 'viaggio', 'minuto', 'ricovero', 'minuto', 'unknown', 'paziente', 'pullman', 'rinvio', 'bottiglia', 'vino', 'acqua', 'bicchiere', 'pasto', 'acqua', 'calice', 'acqua', 'caffe', 'acqua', 'bicchiere', 'acqua', 'vino', 'acqua', 'acqua', 'acqua', 'drink', 'bicchiere', 'acqua', 'birra', 'canzoni', 'brani', 'lacrime', 'unknown', 'inno', 'canzone', 'canzoni', 'latino', 'cover', 'inno', 'lamento', 'tamburi', 'inno', 'canzone', 'piano', '##dito', 'inno', 'inno', 'canzone', 'unknown', 'telefono', 'sindaco', 'skype', 'ministero', 'vigili', 'taxi', 'venditore', 'telefono', 'taxi', 'ragazza', 'dottore', 'dottore', 'telefono', 'telefono', 'gestore', 'taxi', 'macchina', 'provvedimento', 'barca', 'telefono', 'attacco', '##rito', 'combattimenti', 'vendetta', 'battaglie', 'raw', 'diavolo', 'ettore', 'unknown', 'tigre', 'unknown', 'volontario', 'nemici', 'alieni', 'terremoto', 'wrestling', 'lotta', 'battaglia', 'concorrenza', 'unknown', 'professor', 'giornalista', 'rubrica', 'conduttore', 'conduttore', 'cantautore', 'conduttore', 'conduttore', 'conduttore', 'maestro', 'conduttore', 'giornalista', 'giornalista', 'professor', 'trasmissione', 'conduttore', 'relatrice', 'giornalista', 'conduttore', 'curatore', 'ricchezza', 'denaro', 'benzina', 'energia', 'cibi', 'consumi', 'energia', 'consumi', 'energia', 'calorie', 'energia', 'carburante', 'calorie', 'pasto', 'energia', 'carburante', 'cibo', 'carburante', 'euro', 'energia', 'rischi', 'stagione', 'domenica', 'prototipi', 'gara', 'unknown', 'corse', 'unknown', 'sport', 'volontario', 'olimpiade', 'domenica', 'riscatto', 'distanza', 'rischi', 'unknown', 'sprint', 'unknown', 'campionato', 'maratona', 'frase', 'pane', 'cibi', 'cibi', 'verdure', 'ricetta', 'pasto', 'spaghetti', 'barbecue', 'piatti', 'hamburger', 'pane', 'pane', 'unknown', 'pollo', 'barbecue', 'pasta', 'carne', 'legna', 'pesce', 'maestro', 'pianoforte', 'dialoghi', 'pellicole', 'produttori', 'film', 'film', 'spettacoli', 'coro', 'coro', 'doppiaggio', 'maestro', 'regista', 'pellicola', 'regista', 'pianista', 'coro', 'film', 'gara', 'musical', 'disegno', 'fumetto', 'foglio', 'fumetto', 'ritratto', 'fumetto', 'quadri', 'orso', 'personaggi', 'disegno', 'storie', 'quadro', 'disegni', 'equazioni', 'fumetto', 'personaggio', 'elemento', 'ritratti', 'personaggio', 'personaggio', 'tabacco', 'tabacco', 'caffe', 'sigaretta', 'sigarette', 'sostanza', 'sigaretta', 'marijuana', 'tabacco', 'birra', 'sigaretta', 'tabacco', 'sigaretta', 'sigarette', 'tabacco', 'sigaretta', 'caffe', 'caffe', 'sigaretta', 'sigarette', 'domenica', 'partita', 'debutto', 'sera', 'tempo', 'campionato', 'online', 'online', 'ruolo', 'partita', 'partita', 'unknown', 'domenica', 'ruolo', 'partita', 'partita', 'unknown', 'torneo', 'unknown', 'ruolo', 'guadagno', 'profitti', 'guadagno', 'soldi', 'denaro', 'denaro', 'guadagno', 'guadagno', 'guadagni', 'trofeo', 'pubblicitari', 'soldi', 'denaro', 'guadagni', 'reputazione', 'online', 'fiducia', 'licenza', 'soldi', 'guadagno', 'taxi', 'mondo', 'veicolo', 'alcolici', 'gomme', 'veicolo', 'veicoli', 'strada', 'veicoli', 'divieto', 'unknown', 'vettura', 'ragazzi', 'dritto', 'veicoli', 'veicoli', 'cielo', 'unknown', 'camion', 'veicolo', 'testo', 'brano', 'testi', 'blog', 'messaggio', 'giornale', 'libro', 'parole', 'poesie', 'storie', 'volta', 'giornale', 'giornali', 'giornale', 'testo', 'gente', 'articoli', 'albo', 'righe', 'resoconto', 'cuccioli', 'vitello', 'pane', 'panino', 'carne', 'pesci', 'grassi', 'panino', 'panino', 'pane', 'panino', 'piatto', 'pesce', 'pizza', 'salumi', 'unknown', 'pizza', 'birra', 'panino', 'pane', '##mail', 'birra', 'caffe', 'pasto', 'catalogo', 'caffe', 'menu', 'prodotto', 'prodotto', 'prodotti', 'gioco', 'articolo', 'corsi', 'cocktail', 'alcolici', 'drink', 'cocktail', 'hamburger', 'piatti', 'caffe', 'soldo', 'tasse', 'tasse', 'canone', 'prezzo', 'debito', 'tasse', 'biglietto', 'morte', 'riscatto', 'riscatto', 'perdite', 'violenza', 'dazio', 'tributo', '##ndone', 'prezzo', 'biglietto', 'danni', 'battaglie', 'partita', 'pareggio', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'cappotto', 'partita', 'soldo', 'coraggio', 'democrazia', 'partita', 'salvezza', 'partita', 'partita', 'partita', 'timone', 'bussola', 'minuto', 'preghiere', 'giorno', 'preghiera', 'unknown', 'unknown', 'preghiere', 'signore', 'preghiera', 'geova', 'cittadinanza', 'unknown', 'utente', 'unknown', 'cliente', 'lettore', 'lettore', 'sindaco', 'amici', 'docente', 'medici', 'paese', 'unknown', 'rischio', 'giuria', 'iniziativa', '##more', 'comparto', 'gente', 'paziente', 'governo', 'mente', 'unknown', 'fenomeno', 'cittadini', 'rischio', 'puntata', 'unknown', 'unknown', 'preoccupazione', 'biglietti', 'sinfonia', 'biglietti', 'pista', 'volta', 'chitarra', 'tour', 'musica', 'appartamento', 'chitarra', 'sensazioni', 'musica', 'prove', 'idea', 'spettacolo', 'costumi', 'gioco', 'emozione', 'salto', 'musical', 'aria', 'aria', 'mare', 'fuoco', 'respiro', 'aria', 'sensazione', 'aria', 'aria', 'respiro', 'ossigeno', '##tore', 'respiro', 'urla', 'paura', 'unknown', 'liquido', 'balena', 'pelle', 'odore', 'testo', 'poema', 'riga', 'amico', 'online', 'articolo', 'commento', 'articolo', 'discorso', 'articolo', 'articolo', 'lettera', 'libro', 'poesie', 'righe', 'canzone', 'mail', 'righe', 'poesie', 'libro', 'unknown', 'cameriera', 'sedute', 'giocatori', 'cameriera', 'tavoli', 'unknown', 'modelle', 'piatti', 'vino', 'fianco', 'piatto', 'piatto', 'pollo', 'salsa', 'banchetto', 'torta', 'cottura', 'unknown', 'ore', 'pezzi', 'folk', 'concerto', 'canzone', 'secco', 'disco', 'tamburi', 'pianoforte', 'canzone', 'canzone', 'campanello', 'concerto', 'canzoni', 'chitarra', 'note', 'campane', 'campanello', 'tamburi', 'campanello', 'corno', 'corda', 'rigore', 'somme', 'tiro', 'pallone', 'tiro', 'pallone', 'palla', 'tiro', 'tiro', 'calci', 'dritto', 'pallone', 'tiro', 'dritto', 'palla', 'pallone', 'tiro', 'calci', 'palla', 'lotteria', 'trofeo', 'edizione', 'elezioni', 'torneo', 'gara', 'scommessa', 'soldi', 'partita', 'premio', 'derby', 'round', 'partita', 'gara', 'competizione', 'campionato', 'resistenza', 'battaglia', 'battaglia', 'vittoria']
electra = ['video', 'silenzio', 'protagonista', 'brano', 'concerto', 'testo', 'musica', 'voce', 'musica', 'lezioni', 'senatore', 'voce', 'canto', 'voce', 'suono', 'messaggio', 'pianoforte', 'occhi', 'video', 'respiro', 'attimo', 'riunione', 'giorno', 'festa', 'tempo', 'rientro', 'anno', 'risposta', 'momento', 'settimana', 'messa', 'atterraggio', 'vita', 'messa', 'morte', 'voce', 'traguardo', 'medico', 'treno', 'sospensione', 'nome', 'birra', 'latte', 'vino', 'latte', 'vino', 'sangue', 'caffè', 'caffè', 'caffè', 'vino', 'birra', 'vino', 'pane', 'coppa', 'vino', 'caffè', 'vino', 'caffè', 'caffè', 'protagonista', 'brano', 'canzone', 'unknown', 'ritmo', 'brano', 'coro', 'concerto', 'musical', 'inno', 'canzone', 'testo', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'brano', 'unknown', 'venditore', 'sindaco', 'signora', 'venditore', 'medico', 'medico', 'busta', 'amico', 'taxi', 'ragazza', 'maestra', 'medico', 'telefono', 'dottori', 'venditore', 'telefono', 'padre', 'legge', 'capitano', 'numero', 'unknown', 'unknown', 'guerra', 'rivolta', 'battaglia', 'vita', 'battaglia', 'nemico', 'violenza', 'cancro', 'violenza', 'battaglia', 'unknown', 'guerra', 'paura', 'nemico', 'lotta', 'match', 'panico', 'battaglia', 'dottor', 'giornalista', 'giornalista', 'coach', 'regista', 'attore', 'maestro', 'giornalista', 'giornalista', 'maestro', 'regista', 'giornalista', 'scrittrice', 'professor', 'serie', 'regista', 'scuola', 'giornalista', 'collega', 'regista', 'lavoro', 'denaro', 'motore', 'pianeta', 'moda', 'carburante', 'ricchezza', 'peso', 'cibo', 'vita', 'cibo', 'carburante', 'cibo', 'denaro', 'energia', 'carburante', 'cibo', 'benzina', 'carburante', 'carburante', 'gara', 'contratto', 'traguardo', 'gara', 'domenica', 'traguardo', 'titolo', 'unknown', 'mondo', 'vita', 'campionato', 'domenica', 'gara', 'distanza', 'tensione', 'unknown', 'unknown', 'comando', 'basket', 'notizia', 'piatto', 'pasta', 'cibo', 'dolci', 'forno', 'online', 'piatto', 'piatto', 'bambini', 'pesce', 'piatto', 'panino', 'pane', 'pesce', 'panino', 'cibo', 'pane', 'pasta', 'cibi', 'piatto', 'coro', 'suono', 'scena', 'film', 'film', 'film', 'personaggio', 'tenore', 'concerto', 'violino', 'cast', 'coro', 'regista', 'film', 'presidente', 'tenore', 'spedizione', 'film', 'programma', 'film', 'scuola', 'libro', 'foglio', 'giorno', 'quadro', 'computer', 'disegno', 'orso', 'persone', 'disegno', 'volume', 'vita', 'pittura', 'piano', 'fumetto', 'quadro', 'composizione', 'ritratto', 'sceneggiatura', 'fumetto', 'fumo', 'fumo', 'sacco', 'sigaretta', 'giorno', 'sigarette', 'sigaretta', 'fumo', 'tabacco', 'tabacco', 'fumo', 'sigaretta', 'sigaretta', 'tabacco', 'tabacco', 'sigaretta', 'tabacco', 'uomo', 'sigaretta', 'sigaretta', 'pallone', 'partita', 'partita', 'partita', 'ruolo', 'gioco', 'online', 'online', 'calcio', 'partita', 'partita', 'calcio', 'match', 'difesa', 'match', 'match', 'partita', 'unknown', 'set', 'campionato', 'software', 'denaro', 'bonus', 'lavoro', 'tempo', 'denaro', 'titolo', 'soldo', 'metà', 'giocatore', 'capitale', 'lavoro', 'coca', 'stipendio', 'lavoro', 'online', 'centesimo', 'vita', 'soldo', 'soldi', 'macchina', 'mondo', 'veicolo', 'veicolo', 'macchina', 'strada', 'strada', 'vita', 'strada', 'strada', 'ragazzo', 'mondo', 'ragazzi', 'strada', 'veicolo', 'veicoli', 'macchina', 'strada', 'macchina', 'unknown', 'testo', 'libro', 'commenti', 'libro', 'messaggio', 'giornale', 'profeta', 'libro', 'unknown', 'libro', 'libro', 'libro', 'libro', 'sole', 'parola', 'libro', 'dati', 'articolo', 'notizie', 'video', 'cani', 'latte', 'carne', 'pesce', 'latte', 'pesce', 'cibo', 'pesce', 'panino', 'cibo', 'panino', 'pane', 'pesce', 'panino', 'formaggi', 'lavoratori', 'pizza', 'fagioli', 'pranzo', 'panino', 'domanda', 'prodotto', 'cena', 'mani', 'spedizione', 'caffè', 'conto', 'servizio', 'prodotto', 'prodotto', 'gioco', 'prodotto', 'Corso', 'caffè', 'caffè', 'caffè', 'caffè', 'pranzo', 'ristorante', 'caffè', 'matrimonio', 'risarcimento', 'debito', 'silenzio', 'cane', 'debito', 'tassa', 'biglietto', 'vita', 'lettera', 'riscatto', 'contratto', 'pena', 'prezzo', 'potere', 'unknown', 'unknown', 'prezzo', 'prezzo', 'prezzo', 'partita', 'vantaggio', 'partita', 'partita', 'partita', 'partita', 'partita', 'vita', 'anno', 'titolo', 'gioco', 'vita', 'pace', 'promozione', 'salvezza', 'promozione', 'campionato', 'partita', 'leadership', 'memoria', 'unknown', 'scuse', 'Signore', 'Dio', 'Dio', 'papa', 'vita', 'messa', 'preghiera', 'domenica', 'Signore', 'gente', 'Cliente', 'segreteria', 'prenotazione', 'redazione', 'cortesia', 'Sindaco', 'Sindaco', 'cittadinanza', 'medico', 'popolazione', 'squadra', 'rischio', 'giuria', 'questione', 'governo', 'consumatore', 'lettore', 'paziente', 'Governo', 'mondo', 'tema', 'cittadini', 'Paese', 'caso', 'situazione', 'unknown', 'importanza', 'città', 'materiale', 'musica', 'materiale', 'preparazione', 'birra', 'musica', 'band', 'musica', 'casa', 'band', 'musica', 'musica', 'lezione', 'provino', 'concerto', 'musica', 'unknown', 'ritmo', 'musica', 'scena', 'lingua', 'corpo', 'natura', 'contatto', 'mondo', 'unknown', 'corpo', 'vita', 'respiro', 'corpo', 'cellule', '##re', 'corpo', 'voce', 'corpo', 'corpo', 'corpo', 'natura', 'respiro', 'odore', 'programma', 'storia', 'articolo', 'articolo', 'unknown', 'articolo', 'commento', 'cose', 'testo', 'articolo', 'video', 'messaggio', 'giornale', 'libro', 'lettere', 'messaggio', 'messaggio', 'articolo', 'libro', 'libro', 'piatto', 'pizza', 'sedie', 'bambini', 'cameriera', 'pranzo', 'pizza', 'bambini', 'servizio', 'birra', 'stanza', 'pasta', 'cibo', 'pollo', 'resto', 'messa', 'pasta', 'piatto', 'vino', 'alcolici', 'brani', 'cori', 'jazz', 'chitarra', 'dolore', 'concerto', 'campane', 'pianoforte', 'microfono', 'chitarra', 'sveglia', 'concerto', 'pianoforte', 'jazz', 'pianoforte', 'pianoforte', 'cori', 'biglietti', 'chitarra', 'pianoforte', 'palla', 'pallone', 'molla', 'palla', 'tiro', 'tiro', 'pallone', 'palla', 'tiro', 'pallone', 'pallone', 'pallone', 'gol', 'testa', 'palla', 'pallone', 'palla', 'pallone', 'pallone', 'palla', 'gioco', 'campionato', 'trofeo', 'premio', 'gara', 'gara', 'sinistra', 'gioco', 'partita', '##re', 'campionato', 'partita', 'girone', 'gara', 'unknown', 'carriera', 'guerra', 'battaglia', 'battaglia', 'battaglia']

# riportiamo le annotazioni manuali del primo annotatore (predicted) e del secondo annotatore (predicted2)

predicted = ['complimento', 'suono', 'monologo', 'lezione', 'concerto', 'brani', 'suoni', 'voci', 'tv', 'audio', 'suono', 'rumori', 'suono', 'storia', 'musica', 'respiro', 'musica', 'canzoni', 'radio', 'suoni', 'risposta', 'avvio', 'evento', 'festa', 'caricamento', 'pagamento', 'ferie', 'risposta', 'ritorno', 'aumento', 'persona', 'notizie', 'arrivo', 'turno', 'esito', 'turno', 'risposta', 'esito', 'partenza', 'persone', 'cognac', 'acqua', 'acqua', 'acqua', 'liquidi', 'vino', 'sangue', 'caffè', 'sangue', 'caffè', 'vino', 'acqua', 'vino', 'minestra', 'bevanda', 'acqua', 'alcol', 'vino', 'alcol', 'alcol', 'brani', 'pezzo', 'cinguettio', 'cori', 'cori', 'canzone', 'canzone', 'canzoni', 'canzoni', 'inno', 'canzone', 'canzone', 'canzoni', 'canzoni', 'canzone', 'canti', 'canzoni', 'canzoni', 'salmi', 'cori', 'istituzione', 'amico', 'numero', 'compagnia', 'centralino', 'negozio', 'negozio', 'negozio', 'unknown', 'donna', 'persona', 'Halya', 'compagnia', 'istituzione', 'unknown', 'istituzione', 'donna', 'unknown', 'team', 'numero', 'battaglia', 'incontro', 'battaglie', 'battaglie', 'guerra', 'incontri', 'guerra', 'duello', 'unknown', 'duelli', 'guerra', 'battaglia', 'guerre', 'guerre', 'battaglia', 'incontri', 'guerra', 'match', 'lotta', 'battaglia', 'incontro', 'Forum', 'intervista', 'trasmissione', 'trasmissione', 'spettacolo', 'show', 'trasmissione', 'trasmissione', 'orchestra', 'programma', 'incontro', 'trasmissione', 'incontro', 'Isola dei famosi', 'puntata', 'conferenza', 'incontro', 'interventi', 'trasmissione', 'risorse', 'risorse', 'benzina', 'risorse', 'risorse', 'energia', 'risorse', 'energia', 'calorie', 'risorse', 'energia', 'benzina', 'risorse', 'risorse', 'energia', 'risorse', 'risorse', 'benzina', 'benzina', 'combustibile', 'gara', 'gare', 'gare', 'gare', 'gara', 'gara', 'gare', 'gare', 'gare', 'gare', 'meeting', 'gare', 'gare', 'distanza', 'gara', 'campionato', 'distanza', 'distanza', 'gare', 'gare', 'cena', 'piatto', 'cibo', 'cibo', 'piatto', 'ricette', 'pasto', 'cibo', 'carne', 'piatti', 'pasto', 'pranzo', 'cibo', 'piatto', 'carne', 'cibo', 'cibo', 'pasti', 'cibo', 'cibo', 'appuntamento', 'orchestra', 'film', 'film', 'film', 'film', 'film', 'orchestra', 'concerto', 'orchestra', 'film', 'coro', 'film', 'film', 'film', 'concerto', 'orchestra', 'film', 'orchestra', 'orchestra', 'immagini', 'immagini', 'immagini', 'immagini', 'autoritratto', 'immagini', 'bozzetto', 'immagini', 'immagini', 'immagini', 'immagini', 'pitture', 'immagini', 'figure geometriche', 'immagini', 'immagini', 'immagini', 'ritratti', 'scene', 'fumetti', 'sigarette', 'tabacco', 'sigaro', 'sigaretta', 'sigarette', 'sigarette', 'sigarette', 'tabacco', 'sigarette', 'tabacco', 'sigarette', 'sigarette', 'erba', 'sigarette', 'tabacco', 'sigarette', 'tabacco', 'tabacco', 'tabacco', 'tabacco', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partite', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'match', 'partita', 'gare', 'partite', 'parziale', 'partita', 'soldi', 'soldi', 'soldi', 'soldi', 'denaro', 'denaro', 'soldi', 'soldi', 'soldi', 'soldi', 'denaro', 'soldi', 'soldi', 'denaro', 'soldi', 'soldi', 'soldi', 'soldi', 'soldi', 'soldi', 'automobili', 'veicolo', 'veicolo', 'automobile', 'veicolo sportivo', 'veicolo', 'automobile', 'veicolo', 'veicoli', 'macchina', 'veicolo', 'veicolo', 'macchina', 'veicolo', 'veicolo', 'veicolo', 'autovetture', 'veicolo', 'veicolo', 'automobile', 'testi', 'brano', 'post', 'libri', 'messaggio', 'libro', 'testo', 'storia', 'libro', 'testi', 'libro', 'libro', 'libri', 'testo', 'parole', 'testi', 'articoli', 'post', 'articoli', 'messaggio', 'cibo', 'erba', 'cibo', 'cibo', 'pesci', 'cibo', 'junk food', 'cibo', 'pasto', 'pietanze', 'pasto', 'pasto', 'cibo', 'pasto', 'cibo', 'pranzo', 'pasto', 'pasto', 'cena', 'cena', 'pianta', 'cibo', 'cibo', 'cibo', 'articolo', 'cibo', 'pasto', 'assicurazione', 'lavorazioni', 'prodotti', 'giochi', 'libro', 'corsi', 'cibo', 'cibo', 'cibo', 'cibo', 'cibo', 'cibo', 'cibo', 'parole', 'conseguenze', 'conseguenze', 'conseguenza', 'ingiustizia', 'scelte', 'conseguenze', 'consiglio', 'ingiustizia', 'danno', 'gesto', 'sconfitta', 'conseguenze', 'sforzo', 'colpa', 'errori', 'costo', 'ingiustizia', 'conseguenze', 'prezzo', 'derby', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'gioco', 'guerra', 'partita', 'partita', 'partita', 'campionato', 'partite', 'elezioni', 'dibattito', 'Dio', 'divinità', 'Signore', 'divinità', 'Dio', 'Dio', 'divinità', 'Dio', 'Dio', 'Dio', 'Dio', 'Allah', 'clienti', 'persone', 'persone', 'persone', 'persone', 'interessati', 'sindaco', 'studenti', 'persone', 'persone', 'dirigenti', 'università', 'pubblico', 'persone', 'politici', 'esperti', 'lettore', 'persone', 'cittadini', 'persone', 'esperti', 'persone', 'persone', 'esperti', 'persone', 'persone', 'persone', 'politici', 'album', 'sinfonie', 'brani', 'brani', 'brani', 'concerto', 'concerto', 'canzoni', 'musica', 'brani', 'musica', 'musica', 'concerto', 'spettacolo', 'spettacolo', 'spettacolo', 'parte', 'balletti', 'brani', 'spettacolo', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'ossigeno', 'aria', 'ossigeno', 'ossigeno', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'odore', 'odore', 'parole', 'testi', 'messaggio', 'messaggio', 'messaggi', 'articoli', 'post', 'articoli', 'testo', 'testi', 'documento', 'documento', 'testi', 'testi', 'lettera', 'letterina', 'messaggio', 'lettera', 'testi', 'testi', 'cibo', 'cibo', 'cibo', 'birra', 'cibo', 'cibo', 'pizza', 'cibo', 'cibo', 'bevande', 'cibo', 'cibo', 'avanzo', 'pollo', 'piatto', 'cibo', 'piatto', 'piatto', 'piatto', 'cena', 'musica', 'brani', 'fisarmonica', 'musica', 'chitarra', 'musica', 'brani', 'campanello', 'citofono', 'brani', 'strumento', 'brani', 'musica', 'chitarra', 'musiche', 'strumenti', 'campanello', 'brani', 'musica', 'musica', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'pallone', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'pallina', 'competizione', 'partite', 'competizione', 'elezioni', 'gioco', 'gara', 'elezioni', 'gioco', 'partita', 'partita', 'partita', 'partita', 'partita', 'incontro', 'coppa', 'gare', 'guerra', 'guerra', 'guerra', 'campagna']
predicted2 = ['complimento', 'suono', 'monologo', 'lezione', 'musica', 'brani', 'suono', 'suono', 'tv', 'voce', 'suono', 'suono', 'canto', 'storia', 'audio', 'battito', 'musica', 'musica', 'voce', 'musica', 'risposta', 'transizione', 'evento', 'festa', 'caricamento', 'maggio', 'turno', 'unknown', 'riaccensione', 'unknown', 'persona', 'notizie', 'salvatore', 'turno', 'abate', 'turno', 'risposta', 'morte', 'orario', 'numero', 'cognac', 'acqua', 'acqua', 'acqua', 'liquido', 'vino', 'sangue', 'caffè', 'sangue', 'caffè', 'vino', 'acqua', 'vino', 'liquido', 'liquido', 'acqua', 'alcolici', 'vino', 'alcolici', 'alcolici', 'canzoni', 'pezzo', 'melodia', 'coro', 'coro', 'canzone', 'canzoni', 'canzone', 'canzoni', 'canzoni', 'canzone', 'canto', 'canzone', 'canzone', 'canzoni', 'canzoni', 'canzoni', 'canzoni', 'coro', 'coro', 'istituzione', 'persona', 'cliente', 'assicurazione', 'Centrale', 'negozio', 'persona', 'servizio', 'unknown', 'donna', 'persona', 'persona', 'servizio', 'servizio', 'istituzione', 'servizio', 'lei', 'istituzione', 'team', 'numero', 'nemico', 'avversario', 'guerra', 'guerre', 'guerra', 'incontro', 'battaglia', 'duello', 'unknown', 'scontro', 'guerra', 'battaglia', 'guerre', 'guerra', 'scontro', 'competizione', 'guerra', 'incontro', 'difficoltà', 'battaglia', 'programma TV', 'forum', 'programma TV', 'meeting', 'programma TV', 'programma TV', 'porta a porta', 'Fùtbol', 'trasmissione', 'ottoni', 'programma TV', 'evento', 'unknown', 'attività', 'isola dei famosi', 'puntata', 'attività', 'incontro', 'programma radiofonico', 'attività', 'risorse', 'risorse', 'carburante', 'risorse', 'risorse', 'energia', 'risorse', 'energia', 'calorie', 'risorse', 'energia', 'carburante', 'risorse', 'risorse', 'energia', 'risorse', 'risorse', 'carburante', 'carburante', 'carburante', 'competizione', 'gare', 'competizione', 'competizione', 'gara', 'competizione', 'gara', 'gara', 'gare', 'gare', 'gara', 'gara', 'competizione', 'gara', 'corsa', 'corsa', 'corsa', 'Trials', 'corsa', 'competizione', 'cena', 'birra', 'cibo', 'cibo', 'cibo', 'ricetta', 'pasto', 'peperone', 'grigliate', 'pesce', 'pasti', 'pranzo', 'cibo', 'cibo', 'grigliate', 'cibo', 'cibo', 'cibo', 'cibo', 'cibo', 'coro', 'performace musicali', 'film', 'film', 'film', 'film', 'film', 'performance', 'Clarinet Choir Amiata', 'orchestra', 'performance musicale', 'coro', 'film', 'film', 'film', 'concerto', 'performance musicale', 'film', 'performance musicale', 'orchestra', 'immagine', 'immagine', 'immagine', 'immagine', 'autoritratto', 'immagine', 'figure', 'immagine', 'immagine', 'immagine', 'immagine', 'pittura rupestre', 'immagini', 'immagine', 'vignette', 'fumetti', 'immagine', 'immagini', 'scene', 'fumetti', 'tabacco', 'sostanza', 'sigari', 'sostanza', 'sigarette', 'tabacco', 'tabacco', 'sostanza', 'sostanza', 'tabacco', 'sostanza', 'tabacco', 'droga', 'tabacco', 'sostanza', 'sigarette', 'tabacco', 'tabacco', 'sostanza', 'tabacco', 'partita', 'gara', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'match', 'partita', 'competizione', 'competizione', 'partita', 'partita', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'denaro', 'automobili', 'veicoli', 'veicolo', 'veicoli', 'veicoli', 'veicolo', 'automobili', 'veicolo', 'veicoli', 'macchina', 'veicolo', 'auto', 'veicoli', 'veicolo', 'veicolo', 'autovetture', 'veicolo', 'auto', 'veicolo', 'automobile', 'libro ', 'testo', 'thread', 'libri', 'messaggio', 'testo', 'testo ', 'racconto', 'testi', 'testi', 'libro', 'documento', 'testi', 'testi', 'parole', 'testi', 'testi', 'testo', 'documenti', 'resoconto', 'cibo', 'cibo', 'cibo', 'cibo', 'cibo', 'pesce', 'junk food', 'cibo', 'pasti', 'pietanze', 'pasto', 'pasto', 'cibo', 'pasto', 'cibo', 'pasto', 'cibo', 'pasto', 'cena', 'cena', 'animali', 'cibo', 'cibo', 'cibo', 'unknown', 'pasto', 'pasto', 'assicurazione', 'prodotti', 'prodotti', 'giochi', 'libro', 'corsi', 'cibo', 'cibo', 'cibo', 'cibo', 'pasti', 'cibo', 'cibo', 'errore', 'colpa', 'unknown', 'prezzo', 'prezzo', 'default', 'prezzo', 'prezzo', 'opposizione', 'errore', 'salvataggio', 'colpa', 'prezzo', 'guadagni', 'fio', 'debolezza', 'valore', 'colpa', 'presenza', 'prezzo', 'derby', 'partita', 'partite', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'partita', 'gioco', 'guerra', 'partita', 'partita', 'partita', 'partite', 'partita', 'elezioni', 'unknown', 'Dio', 'divinità', 'Dio', 'divinità', 'divinità', 'Dio', 'divinità', 'Dio', 'Dio', 'YHWH', 'Dio', 'Allah', 'utenti', 'richiedente', 'utenti', 'utente', 'utente', 'responsabile', 'sindaco', 'discenti', 'popolazione', 'popolazione', 'allenatori', 'comunità accademica', 'interessati', 'interessati', 'popolazione', 'interessati', 'interessati', 'ammalati', 'interessati', 'popolazione', 'interessati', 'popolazione', 'avversari', 'interessati', 'popolazione', 'interessati', 'interessati', 'autorità', 'album', 'musica ', 'brani', 'brani', 'brani', 'concerto', 'brani', 'brani', 'musica ', 'musica ', 'musica', 'performance', 'performance', 'spettacolo', 'spettacolo', 'spettacolo', 'parte', 'ballo', 'canzoni', 'spettacolo', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria ', 'aria ', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'aria', 'odore ', 'odore ', 'testo', 'testo', 'mail', 'messaggio', 'post', 'articoli', 'articoli', 'articoli', 'testi', 'articoli', 'testo', 'testo', 'testo', 'testi', 'lettera', 'lettera', 'mail', 'mail', 'testi', 'testi', 'pasto', 'cibo', 'pasti', 'boccali', 'pasti', 'pasti', 'pizze', 'cibo', 'pasti', 'cibo', 'pasti', 'cena', 'avanzi', 'petto di pollo', 'piatto', 'bevande', 'piatto', 'piatto', 'piatto', 'cibo', 'pezzi', 'musica', 'jazz', 'musica', 'strumento', 'musica', 'musica', 'campanello', 'campanello', 'musica', 'strumento', 'musica', 'musica', 'chitarra', 'musica', 'musica', 'campanello', 'musica', 'musica', 'musica', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'pallone', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'unknown', 'competizione', 'partita', 'competizione', 'elezioni', 'gioco', 'gara', 'elezioni', 'gioco', 'partita', 'partita', 'partita', 'partita', 'partita', 'gara', 'slalom', 'gara', 'guerra', 'guerra', 'guerra', 'elezioni']

In [ ]:
import re # importiamo la libreria per le espressioni regolari

models = [bertm, bert_it, umberto, bertino, electra]
for model in models: # iteriamo sul risultato di ogni modello
  output = model # l'ouput è la lista di parole contenuta nel nome del modello

  def calculate_similarity(word1, word2): # definiamo la funzione per calcolare la similarity
    word1 = re.sub(r"_\d+", "", word1) # per evitare sovrapposizioni, aggiungiamo una numerazione alle parole ripetute e la rimuoviamo in seguito
    word2 = re.sub(r"_\d+", "", word2)
    word1_token = nlp(word1)
    word2_token = nlp(word2)
    if word2 == "unknown":
        return "unknown"
    if word1_token.has_vector and word2_token.has_vector:
        similarity = word1_token.similarity(word2_token)
        similarity_rounded = round(similarity, 2)
        return similarity_rounded
    else:
        return "unknown"

  # similarity annotatore 1

  word_dict = {} # creiamo un dizionario che associa le coppie dell'annotazione manuale e dell'annotazione automatica
  counter = {}
  for predicted_word, output_word in zip(predicted, output):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word

  similarity_scores = [] # calcoliamo la similarity per ogni coppia nel dizionario
  for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores.append(similarity)
  print('similarity_annotatore1 = ', similarity_scores)

  # similarity annotatore 2

  word_dict = {}
  counter = {}
  for predicted_word, output_word in zip(predicted2, output):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word
  similarity_scores2 = []
  for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores2.append(similarity)
  print('similarity_annotatore2 =', similarity_scores2)

  # media delle due similarity

  similarity_scores_mean = []
  for num1, num2 in zip(similarity_scores, similarity_scores2):
    if isinstance(num1, (int, float)) and isinstance(num2, (int, float)):
      media = (num1 + num2) / 2
      similarity_scores_mean.append(round(media,2))
    else:
      similarity_scores_mean.append('unknown')
  print('similarity_media =', similarity_scores_mean)

KeyboardInterrupt: 

**Terzo step:** lemmatizzazione delle annotazioni manuali e automatiche e calcolo delle similarity

In [ ]:
# lemmatizzazione delle annotazioni manuali

predicted_lemmatized = []
for i in predicted: # annotatore1
    single = [] # creiamo una lista in cui inserire gli elementi di una multiword expression appartenenti ad un'unica annotazione
    doc = nlp(i)
    for token in doc:
      single.append(token.lemma_)
    predicted_lemmatized.append(" ".join(single))
print('predicted_lemmatized =', predicted_lemmatized)
predicted_lemmatized2 = []
for i in predicted2: # annotatore2
    single = []
    doc = nlp(i)
    for token in doc:
      single.append(token.lemma_)
    predicted_lemmatized2.append(" ".join(single))
print('predicted_lemmatized2 =', predicted_lemmatized2)

# lemmatizzazione delle annotazioni automatiche e calcolo della similarity con gli annotatori

for model in models:
  lemmatized = []
  for i in model:
    single = []
    doc = nlp(i)
    for token in doc:
      single.append(token.lemma_)
    lemmatized.append(" ".join(single))
  print(lemmatized)

  # similarity annotatore1

  word_dict = {}
  counter = {}

  for predicted_word, output_word in zip(lemmatized, predicted_lemmatized):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word
  print(len(word_dict))
  similarity_scores = []
  for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores.append(similarity)
  print('similarity_predicted1 =', similarity_scores)

  # similarity annotatore2

  word_dict = {}
  counter = {}

  for predicted_word, output_word in zip(lemmatized, predicted_lemmatized2):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word
  print(len(word_dict))
  similarity_scores2 = []
  for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores2.append(similarity)
  print('similarity_predicted2 =', similarity_scores2)

  # media delle due similarity

  similarity_scores_mean = []
  for num1, num2 in zip(similarity_scores, similarity_scores2):
    if isinstance(num1, (int, float)) and isinstance(num2, (int, float)):
      media = (num1 + num2) / 2
      similarity_scores_mean.append(round(media,2))
    else:
      similarity_scores_mean.append('unknown')
  print('similarity_media =', similarity_scores_mean)

predicted_lemmatized = ['complimento', 'suono', 'monologo', 'lezione', 'concerto', 'brano', 'suono', 'voce', 'tv', 'audio', 'suono', 'rumore', 'suono', 'storia', 'musica', 'respiro', 'musica', 'canzone', 'radio', 'suono', 'risposta', 'avvio', 'evento', 'festa', 'caricamento', 'pagamento', 'feria', 'risposta', 'ritorno', 'aumento', 'persona', 'notizia', 'arrivo', 'turno', 'esito', 'turno', 'risposta', 'esito', 'partenza', 'persona', 'cognac', 'acqua', 'acqua', 'acqua', 'liquido', 'vino', 'sangue', 'caffè', 'sangue', 'caffè', 'vino', 'acqua', 'vino', 'minestra', 'bevanda', 'acqua', 'alcol', 'vino', 'alcol', 'alcol', 'brano', 'pezzo', 'cinguettio', 'core', 'core', 'canzone', 'canzone', 'canzone', 'canzone', 'inno', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'canto', 'canzone', 'canzone', 'salmo', 'core', 'istituzione', 'amico', 'numero', 'compagnia', 'centralino', 'negozio', 'negozio', 'negozio', 'unknown', 'donna', 'persona', 'Halya', 'compagnia', 'istituzione', 'unknown', 'i